1. 문서의 내용을 읽고
2. 문서를 쪼갠다
- 토큰 수 초과로 답변을 생성하지 못하거나
- 문서가 길면 (인풋) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 DB에 저장
4. 질문이 있을 때 벡터 DB 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달하는 식으로 구현

In [ ]:
%pip install --upgrade docx2txt langchain-community

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
%pip install -qU langchain-chroma

In [ ]:
%pip install -qU langchain langchainhub

In [ ]:
%pip install -qU langchainhub

In [ ]:
%pip install -U langchain langchain-community langchain-core langchain-chains

In [13]:
%pip install -U langchain langchain-community langchain-core

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax_with_table.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [2]:
len(document_list)

225

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [4]:
from langchain_chroma import Chroma

#database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name="chroma-tax",persist_directory="./chroma")
database = Chroma(collection_name="chroma-tax", persist_directory="./chroma", embedding_function=embedding)

In [5]:
query = "연봉 5천만원의 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3)

In [6]:
retrieved_docs

[Document(id='82b07f71-2d3b-4d49-9631-51e2f65d5bcc', metadata={'source': './tax_with_table.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제1

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [8]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [9]:
ai_message = llm.invoke(prompt)

In [10]:
ai_message.content

'연봉 5천만원의 직장인의 소득세를 계산하기 위해서는 기본 세율을 적용하여야 합니다. 한국의 소득세는 누진 과세 방식으로 적용되며, 각 구간마다 다른 세율이 적용됩니다. 2023년 기준으로 적용되는 소득세 기본세율 구간은 다음과 같습니다.\n\n- 1,200만원 이하: 6%\n- 1,200만원 초과 ~ 4,600만원 이하: 15%\n- 4,600만원 초과 ~ 8,800만원 이하: 24%\n- 8,800만원 초과 ~ 1억 5천만원 이하: 35%\n- 1억 5천만원 초과 ~ 3억원 이하: 38%\n- 3억원 초과 ~ 5억원 이하: 40%\n- 5억원 초과: 42%\n\n연봉 5천만원인 경우, 이를 각 구간에 따라 다음과 같이 계산할 수 있습니다.\n\n1. 1,200만원 이하 금액: 1,200만원 × 6% = 72만원\n2. 1,200만원 초과 ~ 4,600만원 이하 금액: (4,600만원 - 1,200만원) × 15% = 510만원\n3. 4,600만원 초과 ~ 5,000만원 이하 금액: (5,000만원 - 4,600만원) × 24% = 96만원\n\n따라서 총 소득세는 72만원 + 510만원 + 96만원 = 678만원입니다.\n\n이 계산에는 기초공제나 세액공제, 또는 기타 개인의 상황에 따라 적용될 수 있는 공제는 반영되지 않았습니다. 정확한 세액을 산출하기 위해서는 개인의 공제사항 등을 반영하여야 합니다.'

In [11]:
load_dotenv()

from langsmith import Client
client = Client()
prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

In [12]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 프롬프트 (LangSmith에서 가져온 prompt 사용 또는 직접 작성)
# LangSmith prompt를 사용하려면:
from langsmith import Client
client = Client()
prompt = client.pull_prompt("rlm/rag-prompt")

retriever = database.as_retriever()

# RAG 체인 구성
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

# 실행
ai_message = qa_chain.invoke(query)
print(ai_message)

연봉 5천만원의 근로소득이 있는 직장인의 소득세는 기본세율을 적용받습니다. 그러나 기본세율이 실제 세금 금액을 의미하지 않기 때문에 정확한 소득세 금액은 공제항목과 기타 개인적인 요인에 따라 달라질 수 있습니다. 따라서 명확한 금액을 계산하려면 자세한 세율표와 공제 항목 등을 참조해야 합니다.
